In [3]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from statsmodels.nonparametric.kernel_density import KDEMultivariate
# from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import StratifiedGroupKFold
import seaborn as sns
import numpy as np
import re
import os
import sys; print(sys.executable)

/Users/gc3045/miniconda3/bin/python


In [4]:

%load_ext autoreload
%autoreload 2

from helpers.utils import learn_kde
from helpers.utils import score_with_kdes
from helpers.utils import reshape_to_cmat, _site_slice
import helpers.utils 

In [5]:
codebook_fname = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/PEtracer raw data/edit_codebook.csv"
validation_fname = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/PEtracer raw data/preedited_merfish_invitro_alleles.csv"
colony_fname = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/PEtracer raw data/colony_alleles.csv"

out_dir = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/inputs/"


In [7]:
validation_df = pd.read_csv(validation_fname)
codebook = pd.read_csv(codebook_fname)

train_df = validation_df.copy().fillna("unedited")
codebook_df = codebook.copy().fillna("unedited")
colony_df = pd.read_csv(colony_fname, index_col=0).fillna("unedited")

In [8]:
id_cols    = ["cellBC", "intID", "clone"]
pieces = [
    _site_slice(train_df, "HEK3", id_cols, codebook_df),
    _site_slice(train_df, "EMX1", id_cols, codebook_df),
    _site_slice(train_df, "RNF2", id_cols, codebook_df),
]
long_train_df = pd.concat(pieces, axis=0, ignore_index=True)

In [9]:
train_df

,intID,cellBC,HEK3,EMX1,RNF2,spot,x,y,z,global_x,...,cell_dist,sample,clone,whitelist,RNF2_actual,HEK3_actual,EMX1_actual,RNF2_brightest,HEK3_brightest,EMX1_brightest
0,intID1011,4T1_preedited-1057,CTCTC,AGTAC,ACTCC,226,504,123,20,-765.072,...,0.139511,4T1_preedited,2,True,ACTCC,CTCTC,AGTAC,ACTCC,CTCTC,AGTAC
1,intID1011,4T1_preedited-1120,CTCTC,AGTAC,ACTCC,42,1928,1712,26,-612.704,...,1.070000,4T1_preedited,2,True,ACTCC,CTCTC,AGTAC,ACTCC,CTCTC,AGTAC
2,intID1011,4T1_preedited-1143,CTCTC,AGTAC,ACTCC,105,1951,952,31,-610.243,...,0.348657,4T1_preedited,2,True,ACTCC,CTCTC,AGTAC,ACTCC,CTCTC,AGTAC
3,intID1011,4T1_preedited-1153,CTCTC,AGTAC,ACTCC,191,2189,647,23,-584.777,...,1.533115,4T1_preedited,2,True,ACTCC,CTCTC,AGTAC,ACTCC,CTCTC,AGTAC
4,intID1011,4T1_preedited-1162,CTCTC,AGTAC,ACTCC,11,1944,1198,6,-610.992,...,3.000000,4T1_preedited,2,True,ACTCC,CTCTC,AGTAC,ACTCC,CTCTC,AGTAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43930,intID974,4T1_preedited-8630,GCGCC,ATTCG,TTCCT,631,378,1209,22,1093.446,...,3.472146,4T1_preedited,2,True,TTCCT,GCGCC,ATTCG,TTCCT,GCGCC,ATTCG
43931,intID974,4T1_preedited-8638,GCGCC,ATTCG,TTCCT,158,84,519,31,1061.988,...,2.240381,4T1_preedited,2,True,TTCCT,GCGCC,ATTCG,TTCCT,GCGCC,ATTCG
43932,intID974,4T1_preedited-8639,GCGCC,ATTCG,TTCCT,422,323,533,25,1087.561,...,0.000000,4T1_preedited,2,True,TTCCT,GCGCC,ATTCG,TTCCT,GCGCC,ATTCG
43933,intID974,4T1_preedited-8644,GCGCC,ATTCG,TTCCT,320,431,423,36,1099.117,...,3.177443,4T1_preedited,2,True,TTCCT,GCGCC,ATTCG,TTCCT,GCGCC,ATTCG


In [11]:
mask = colony_df['clone'] == 3
colony_df = colony_df[mask]
print(colony_df['cellBC'].nunique())

832


In [12]:
id_cols    = ["cellBC", "intID", "clone"]

pieces = [
    _site_slice(colony_df, "HEK3", id_cols, codebook_df),
    _site_slice(colony_df, "EMX1", id_cols, codebook_df),
    _site_slice(colony_df, "RNF2", id_cols, codebook_df),
]
long_test_df = pd.concat(pieces, axis=0, ignore_index=True)

In [14]:
unique_cassettes = long_test_df['intID'].unique()
cassette_mapping = {id_: i for i, id_ in enumerate(unique_cassettes)}
long_test_df["cassette_idx"] = long_test_df["intID"].map(cassette_mapping)

site_mapping = {'RNF2': 0, 'HEK3': 1, 'EMX1': 2}
long_test_df['target_idx'] = long_test_df['target_site'].map(site_mapping) + len(site_mapping) * long_test_df["cassette_idx"]

In [16]:
print("Range of target_idx:", min(long_test_df['target_idx']), max(long_test_df['target_idx']))
print("Range of cassette_idx:", min(long_test_df['cassette_idx']), max(long_test_df['cassette_idx']))

Range of target_idx: 0 47
Range of cassette_idx: 0 15


In [17]:
feature_cols = [f"feature_{x}" for x in range(9)]

In [20]:
site_states = (
    long_train_df[['target_site', 'seq_state']]
    .drop_duplicates()
    .groupby('target_site')['seq_state']
    .apply(list)
)

In [21]:
# save a codebook
rows = []
for site, labels in site_states.items():
    # ensure 'unedited' is 0, others get 1..8 sorted alphabetically
    labels_sorted = sorted([l for l in labels if l != "unedited"])
    ordered = ["unedited"] + labels_sorted
    for i, lbl in enumerate(ordered):
        rows.append({"site": site, "label": lbl, "genotype": i})

label_codebook = pd.DataFrame(rows).sort_values(["site", "genotype"], ignore_index=True)
label_codebook.to_csv(out_dir + "/colonies_label_codebook.csv", index=False) 

In [22]:
for col in ['seq', 'brightest', 'pet']:
    long_train_df = long_train_df.merge(
        label_codebook.rename(columns={'site': 'target_site', 'label': f'{col}_state', 'genotype': f'{col}_geno'}),
        on=['target_site', f'{col}_state'],
        how='left'
    )

In [30]:
col = 'pet'
long_test_df = long_test_df.merge(
    label_codebook.rename(columns={'site': 'target_site', 'label': f'{col}_state', 'genotype': f'{col}_geno'}),
    on=['target_site', f'{col}_state'],
    how='left'
)

#### Score

In [32]:
df_train = long_train_df.copy()
y = df_train['seq_geno']
df_test  = long_test_df

In [34]:
feature_cols = [col for col in df_train.columns if col.startswith('feature')]

In [36]:
all_kde_dict_X = {}
all_scales = {}
all_pca = {}
preds_X_full = pd.DataFrame(index=df_test.index, columns=["pred_label","argmax_logpdf"])
scores_X_tables = []

for site in df_train['target_site'].unique():
    print(site)
    site_df_train = df_train.loc[df_train['target_site'] == site]
    site_df_test = df_test.loc[df_test['target_site'] == site]

    all_kde_dict_X[site] = {}
    all_scales[site] = {}
    all_pca[site] = {}
    site_labels = site_df_train["seq_geno"].unique()
    
    for lab in site_labels:
        m_train = site_df_train["seq_geno"] == lab
        print("label", lab, sum(m_train))

        X_block = site_df_train[feature_cols].loc[m_train]        
        scaler = StandardScaler()
        X_block = scaler.fit_transform(X_block.values.copy())
        pca = PCA(n_components=9, random_state=0, svd_solver='full', whiten=False)
        X_block = pca.fit_transform(X_block.copy())
    
        all_scales[site][lab] = scaler
        all_pca[site][lab] = pca
        all_kde_dict_X[site][lab] = learn_kde(X_block)

    # test
    pred_cols = ["pred_label", "argmax_logpdf"]
    collect_logpdf = []
    for lab in site_labels:
        scaler = all_scales[site][lab]
        pca = all_pca[site][lab]
        kde = all_kde_dict_X[site][lab]
        X_test = scaler.transform(site_df_test[feature_cols])
        X_test = pca.transform(X_test)
        jacobian = np.sum(np.log(np.maximum(scaler.scale_, 1e-12)))
        
        logpdf = np.log(np.maximum(kde.pdf(X_test), 1e-300)) - jacobian
        collect_logpdf.append(logpdf)

    L = np.column_stack(collect_logpdf)
    site_labels_arr = np.array(site_labels, dtype=object)
    idx_max = np.argmax(L, axis=1)
    pred_labels = site_labels_arr[idx_max]
    argmax_logpdf = L[np.arange(L.shape[0]), idx_max]
    
    preds_X = pd.DataFrame({
        "pred_label": pred_labels,
        "argmax_logpdf": argmax_logpdf,
    }, index=site_df_test.index)

    scores_X = pd.DataFrame(L, index=site_df_test.index, columns=[f'state{lab}_prob' for lab in site_labels])

    # write predictions into the preallocated frame using the boolean mask
    preds_X_full.loc[df_test['target_site'] == site, pred_cols] = preds_X[pred_cols].to_numpy()
    
    scores_X.index = site_df_test.index
    scores_X = scores_X.add_prefix(f"{site}::")
    scores_X_tables.append(scores_X)
    print("Scored X...")

HEK3
label 4 5491
label 5 10882
label 2 5792
label 0 8780
label 6 5507
label 1 1320
label 7 4166
label 3 642
label 8 1355


/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-

Scored X...
EMX1
label 2 5409
label 1 7873
label 7 6378
label 0 8780
label 8 6083
label 5 4036
label 3 4107
label 6 583
label 4 686


/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-

Scored X...
RNF2
label 2 4160
label 6 6060
label 7 3529
label 0 8780
label 5 3412
label 8 8958
label 4 1248
label 1 5133
label 3 2655


/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/gc3045/miniconda3/lib/python3.13/site-

Scored X...


In [37]:
out = (
    df_test[["intID", "clone", "target_site", "pet_geno"]]
    .join(preds_X_full.add_prefix("X_"))
)

In [38]:
out

,intID,clone,target_site,pet_geno,X_pred_label,X_argmax_logpdf
0,intID1011,3,HEK3,5,5,-56.433557
1,intID1011,3,HEK3,0,0,-52.318358
2,intID1011,3,HEK3,0,0,-52.343101
3,intID1011,3,HEK3,0,0,-67.96673
4,intID1011,3,HEK3,4,4,-53.772232
...,...,...,...,...,...,...
30085,intID911,3,RNF2,0,0,-53.591122
30086,intID911,3,RNF2,0,0,-54.519453
30087,intID911,3,RNF2,8,8,-54.665603
30088,intID911,3,RNF2,0,0,-59.495509


In [39]:
out['X_argmax_logpdf'].astype('float').describe()

count    30090.000000
mean       -65.983467
std         49.872075
min       -740.652463
25%        -62.453945
50%        -57.505158
75%        -54.822460
max        -49.842892
Name: X_argmax_logpdf, dtype: float64

### Save clone 2 to file now

In [40]:
df_clean = df_test.copy()

In [41]:
use_scores = scores_X_tables
use_preds = preds_X_full

In [42]:
df_clean['kde_geno'] = use_preds['pred_label'].astype('Int64')

In [43]:
for i in range(9):
    col = f"state{i}_prob"
    if col not in df_clean.columns:
        df_clean[col] = pd.NA  # initialize

In [44]:
for site, df_scores in zip(df_clean['target_site'].unique(), use_scores):
    idx = df_scores.index  

    for col in df_scores.columns:
        new_col_name = col.split('::')[1]
        df_clean.loc[idx, new_col_name] = df_scores[col].values
        

In [45]:
df_clean

,cellBC,intID,clone,target_site,pet_state,pet_prob,feature_0,feature_1,feature_2,feature_3,...,kde_geno,state0_prob,state1_prob,state2_prob,state3_prob,state4_prob,state5_prob,state6_prob,state7_prob,state8_prob
0,colony-21229,intID1011,3,HEK3,CTTTG,1.000,151.968,382.201,93.477,108.180,...,5,-77.686381,-164.887733,-85.321183,-198.710112,-82.234303,-56.433557,-92.949801,-147.792755,-98.911797
1,colony-21230,intID1011,3,HEK3,unedited,0.993,158.407,176.455,146.892,136.513,...,0,-52.318358,-59.316048,-58.961131,-66.145116,-58.895207,-60.815081,-60.726905,-59.69161,-61.933258
2,colony-21240,intID1011,3,HEK3,unedited,0.950,132.650,127.877,152.962,191.891,...,0,-52.343101,-60.051285,-59.967729,-60.88326,-59.237272,-57.680469,-59.181425,-58.467382,-61.988406
3,colony-21241,intID1011,3,HEK3,unedited,1.000,615.597,1355.207,345.986,404.388,...,0,-67.96673,-742.139,-742.812684,-743.046895,-740.652463,-302.932821,-742.493743,-714.983895,-742.775326
4,colony-21244,intID1011,3,HEK3,CTCTC,1.000,226.663,202.174,126.255,150.680,...,4,-67.088598,-153.104879,-231.964988,-393.707286,-53.772232,-90.444364,-79.078426,-109.032753,-78.389925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30085,colony-22490,intID911,3,RNF2,unedited,0.998,329.336,166.316,234.391,257.896,...,0,-53.591122,-223.47002,-738.11679,-744.39181,-549.394935,-86.006768,-68.724288,-63.426326,-62.340031
30086,colony-22494,intID911,3,RNF2,unedited,0.640,132.163,213.662,211.209,146.451,...,0,-54.519453,-59.181113,-63.428367,-64.154347,-61.389187,-58.586708,-57.351971,-59.310761,-58.25654
30087,colony-22501,intID911,3,RNF2,TTCCT,0.946,237.179,231.872,819.079,149.308,...,8,-66.832212,-61.88085,-60.91954,-62.8047,-77.334496,-66.221017,-72.62434,-65.925372,-54.665603
30088,colony-22504,intID911,3,RNF2,unedited,0.999,855.130,239.156,242.118,455.069,...,0,-59.495509,-79.401966,-738.11679,-744.39181,-743.083796,-86.231949,-72.551742,-78.251196,-76.588391


In [46]:
df_clean.columns

Index(['cellBC', 'intID', 'clone', 'target_site', 'pet_state', 'pet_prob',
       'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'cassette_idx',
       'target_idx', 'pet_geno', 'kde_geno', 'state0_prob', 'state1_prob',
       'state2_prob', 'state3_prob', 'state4_prob', 'state5_prob',
       'state6_prob', 'state7_prob', 'state8_prob'],
      dtype='object')

In [47]:
disagree_mask = df_clean['pet_geno'] != df_clean['kde_geno']

In [49]:
df_clean.loc[disagree_mask, ['cellBC', 'intID', 'clone', 'target_site', 'pet_state', 
       'pet_prob',  
       'pet_geno', 'kde_geno', 'state0_prob', 'state1_prob', 'state2_prob',
       'state3_prob', 'state4_prob', 'state5_prob', 'state6_prob',
       'state7_prob', 'state8_prob']]

,cellBC,intID,clone,target_site,pet_state,pet_prob,pet_geno,kde_geno,state0_prob,state1_prob,state2_prob,state3_prob,state4_prob,state5_prob,state6_prob,state7_prob,state8_prob
72,colony-21357,intID1011,3,HEK3,unedited,1.000,0,2,-77.979126,-272.055533,-70.658201,-377.337272,-246.01975,-136.424338,-213.170198,-103.925939,-637.896869
90,colony-21381,intID1011,3,HEK3,unedited,0.978,0,7,-72.07792,-520.472793,-742.812684,-600.997541,-657.476694,-515.262433,-498.983915,-67.60178,-742.775326
130,colony-21439,intID1011,3,HEK3,GCGCC,0.926,8,0,-66.521859,-217.489662,-199.489576,-743.046895,-147.463342,-126.636336,-144.276835,-377.337888,-116.106343
218,colony-21581,intID1011,3,HEK3,unedited,0.993,0,4,-741.355324,-742.139,-742.812684,-743.046895,-740.652463,-741.578394,-742.493743,-741.8658,-742.775326
336,colony-21805,intID1011,3,HEK3,unedited,0.932,0,8,-64.384227,-102.06537,-64.077499,-77.621753,-83.556841,-74.509013,-94.527233,-69.61829,-63.309456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29198,colony-22075,intID812,3,RNF2,TTCCT,0.983,8,2,-741.597705,-743.857838,-738.11679,-744.39181,-743.083796,-742.929352,-741.484048,-741.97456,-743.245195
29553,colony-21468,intID911,3,RNF2,TTCCT,0.459,8,2,-54.555078,-55.598365,-53.496044,-56.223933,-56.235142,-55.122138,-55.405422,-56.07114,-53.593907
29694,colony-21718,intID911,3,RNF2,GTTCA,0.607,4,8,-69.786562,-68.368814,-80.102562,-73.798529,-64.583235,-75.467777,-74.71282,-71.500543,-62.27555
29945,colony-22192,intID911,3,RNF2,unedited,0.999,0,1,-86.625602,-83.634793,-738.11679,-744.39181,-743.083796,-121.816633,-166.762401,-113.723736,-119.023577


In [52]:
# check that the cassette_idx and target_idx are redone appropriately
# need 3 outputs: 1) petracer geno_df 2) kde_argmax geno 3) kde_scores
df_clean['kde_geno'].nunique(), df_clean['pet_geno'].nunique()

(9, 9)

In [53]:
petracer_id_cols = ["cellBC", "intID", "clone", "target_site", "target_idx"]
petracer_call_cols = ["pet_state", "pet_prob"]

In [58]:
df_clean['target_idx'].nunique(), min(df_clean['target_idx']), max(df_clean['target_idx'])

(48, 0, 47)

In [80]:
df_clean['intID'].nunique()

16

In [103]:
# how many unique cells pass the intID threshold cutoff of 60% 
import math

thresh = 0.5999
total_cassettes = 16
min_intID = thresh * total_cassettes

counts = df_clean.groupby("cellBC")["intID"].nunique()
passed = counts[counts > min_intID]

print(f"{len(passed)} of {len(counts)} cells passed ({len(passed)/len(counts):.1%})")

713 of 832 cells passed (85.7%)


In [118]:
(counts / total_cassettes).mean()

np.float64(0.7534555288461539)

#### Save the full 832 cells

##### Save character matrix

In [60]:
tbl = df_clean[['target_idx', 'cellBC', 'kde_geno']]
tbl.columns = ['target_site', 'cell_name', 'pred_label']
kde_inputs_argmax = reshape_to_cmat(tbl).T

In [61]:
kde_inputs_argmax.to_csv(out_dir + "/colonies_clone3_kde_character_matrix.csv")

##### Save PETracer probabilities now

In [62]:
petracer_df = df_clean.copy()
petracer_path = os.path.join(out_dir, f"colonies_clone3_petracer_genotypes.csv")
petracer_df.to_csv(petracer_path, index=False)

##### Save the kde scores

In [63]:
state_cols = [c for c in df_clean.columns if re.match(r"state\d+_prob", c)]
obs_matrix = df_clean[["cellBC", "cassette_idx", "target_idx"] + state_cols].copy().rename(columns={"cellBC": "cell_name", "target_idx": "target_site"})

obs_matrix["cassette_idx"] = 0
obs_matrix = obs_matrix.sort_values(by="target_site", ascending=True)
petracer_path = os.path.join(out_dir, f"colonies_clone3_kde_scores.csv")
obs_matrix.to_csv(petracer_path, index=False)


In [78]:
obs_matrix['cell_name'].nunique()

832

####  Save the 695 cells that were in the published tree

In [124]:
import treeswift

In [125]:
pet_tree_fname = '/Users/gc3045/git/laml2-experiments/real_data/PEtracer/inputs/trees/colonies/clone3.petracer.neighbor_joining.binary.nwk'
pet_tree = treeswift.read_tree_newick(pet_tree_fname)
pet_leaves = set([n.label for n in pet_tree.traverse_leaves()])

In [127]:
mask = [x in pet_leaves for x in df_clean['cellBC']]
df_subset = df_clean.loc[mask]

In [129]:
df_subset['cellBC'].nunique()

695

In [130]:
# save the character matrix
tbl = df_subset[['target_idx', 'cellBC', 'kde_geno']]
tbl.columns = ['target_site', 'cell_name', 'pred_label']
kde_inputs_argmax = reshape_to_cmat(tbl).T
kde_inputs_argmax.to_csv(out_dir + "/colonies_clone3_kde_character_matrix_subset.csv")

In [133]:
# save petracer probabilities now
petracer_df = df_subset.copy()
petracer_path = os.path.join(out_dir, f"colonies_clone3_petracer_genotypes_subset.csv")
petracer_df.to_csv(petracer_path, index=False)

In [131]:
# save the kde scores
state_cols = [c for c in df_subset.columns if re.match(r"state\d+_prob", c)]
obs_matrix = df_subset[["cellBC", "cassette_idx", "target_idx"] + state_cols].copy().rename(columns={"cellBC": "cell_name", "target_idx": "target_site"})

obs_matrix["cassette_idx"] = 0
obs_matrix = obs_matrix.sort_values(by="target_site", ascending=True)
petracer_path = os.path.join(out_dir, f"colonies_clone3_kde_scores_subset.csv")
obs_matrix.to_csv(petracer_path, index=False)


#### Drop an additional 10% of observations sampled randomly, over 5 iterations

In [64]:
df_clean = df_test.copy()
use_scores = scores_X_tables
use_preds = preds_X_full
df_clean['kde_geno'] = use_preds['pred_label'].astype('Int64')

for i in range(9):
    col = f"state{i}_prob"
    if col not in df_clean.columns:
        df_clean[col] = pd.NA  # initialize

for site, df_scores in zip(df_clean['target_site'].unique(), use_scores):
    idx = df_scores.index  

    for col in df_scores.columns:
        new_col_name = col.split('::')[1]
        df_clean.loc[idx, new_col_name] = df_scores[col].values

petracer_id_cols = ["cellBC", "intID", "clone", "target_site", "target_idx"]
petracer_call_cols = ["pet_state", "pet_prob"]

In [76]:
print(f"We have {df_clean['cellBC'].nunique()} cells and {max(df_clean['target_idx']) - min(df_clean['target_idx'])+1} target sites.")
print(f"For a total of {df_clean['cellBC'].nunique() * (max(df_clean['target_idx']) - min(df_clean['target_idx'])+1)} target sites.")
print(f"We have {df_clean.shape[0]} observations, which covers {round(df_clean.shape[0]/(df_clean['cellBC'].nunique() * (max(df_clean['target_idx']) - min(df_clean['target_idx'])+1)), 3)} of our total.")

We have 832 cells and 48 target sites.
For a total of 39936 target sites.
We have 30090 observations, which covers 0.753 of our total.


In [ ]:
# save character matrix

tbl = df_clean[['target_idx', 'cellBC', 'kde_geno']]
tbl.columns = ['target_site', 'cell_name', 'pred_label']
kde_inputs_argmax = reshape_to_cmat(tbl).T
kde_inputs_argmax.to_csv(out_dir + "/colonies_clone3_kde_character_matrix.csv")

In [ ]:
# save petracer probabilities

petracer_df = df_clean.copy()
petracer_path = os.path.join(out_dir, f"colonies_clone3_petracer_genotypes.csv")
petracer_df.to_csv(petracer_path, index=False)

In [ ]:
# save the kde probabilities

state_cols = [c for c in df_clean.columns if re.match(r"state\d+_prob", c)]
obs_matrix = df_clean[["cellBC", "cassette_idx", "target_idx"] + state_cols].copy().rename(columns={"cellBC": "cell_name", "target_idx": "target_site"})

obs_matrix["cassette_idx"] = 0
obs_matrix = obs_matrix.sort_values(by="target_site", ascending=True)
petracer_path = os.path.join(out_dir, f"colonies_clone3_kde_scores.csv")
obs_matrix.to_csv(petracer_path, index=False)
